# DWEN

In [1]:
import keras

Using TensorFlow backend.


In [2]:
from __future__ import print_function, division

In [3]:
import re
import numpy as np
import pandas as pd

import os
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import sys
from annoy import AnnoyIndex
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
%matplotlib inline

In [4]:
from keras.layers import Conv1D, Input, Add, Activation, Dropout, Embedding, MaxPooling1D, \
    GlobalMaxPool1D, Flatten, Dense, Concatenate, BatchNormalization
from keras.models import Sequential, Model
from keras.regularizers import l2
from keras.initializers import TruncatedNormal
from keras.layers.advanced_activations import LeakyReLU, ELU
from keras import optimizers

In [5]:
from methods.baseline import Baseline
from methods.experiments import Experiment
from methods.evaluation import Evaluation
from methods.retrieval import Retrieval

## Auxiliary methods

## Configurações Globais

In [6]:
MAX_SEQUENCE_LENGTH_T = 20 # 100
MAX_SEQUENCE_LENGTH_D = 20 # 500
EMBEDDING_DIM = 300
MAX_NB_WORDS = 20000
'''
    Configuration
'''
epochs = 1000
freeze_train = .1 # 10% with freeze weights
best_loss = float('inf')
best_epoch = 0
verbose = 0
loss = 0

### Parse bugs preproprecessed

In [7]:
# Domain to use
DOMAIN = 'eclipse'
METHOD = 'baseline_dwen_{}'.format(epochs)
PREPROCESSING = 'bert'
TOKEN = 'bert'
# Dataset paths
DIR = 'data/processed/{}/{}'.format(DOMAIN, PREPROCESSING)
DIR_PAIRS = 'data/normalized/{}'.format(DOMAIN)
DATASET = os.path.join('data/normalized/{}'.format(DOMAIN), '{}.csv'.format(DOMAIN))
# Path embeddings
GLOVE_DIR='data/embed'
# Save model
SAVE_PATH = '{}_preprocessing_{}_feature@number_of_epochs@epochs_64batch({})'.format(PREPROCESSING, METHOD, DOMAIN)
SAVE_PATH_FEATURE = '{}_preprocessing_{}_feature_@number_of_epochs@epochs_64batch({})'.format(PREPROCESSING, METHOD, DOMAIN)

# Extract CORPUs
EXTRACT_CORPUS = False

In [8]:
import os

pretrained_path = 'uncased_L-12_H-768_A-12'
config_path = os.path.join(pretrained_path, 'bert_config.json')
model_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

In [9]:
from keras_bert import load_vocabulary

token_dict = load_vocabulary(vocab_path)

In [10]:
baseline = Baseline(DOMAIN, DIR, DATASET, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D,
                   token_dict['[CLS]'], token_dict['[SEP]'])
evaluation = Evaluation(verbose=0)
retrieval = Retrieval()
experiment = Experiment(baseline, evaluation)

In [11]:
experiment.set_retrieval(retrieval, baseline, DOMAIN)

#### Loading bug ids in memory

In [12]:
experiment.load_ids()
len(baseline.bug_ids)

Reading bug ids


361006

#### Dicionário de títulos e descrições

In [13]:
%%time

experiment.load_bugs(TOKEN)
len(baseline.sentence_dict)


CPU times: user 34.9 s, sys: 3.7 s, total: 38.6 s
Wall time: 37.9 s


#### Hashing bugs by buckets

In [14]:
issues_by_buckets = experiment.get_buckets_for_bugs()

#### Prepare the train and test

In [15]:
experiment.prepare_dataset(issues_by_buckets, path_train='train_chronological', path_test='test_chronological')
# Read and create the test queries duplicates
retrieval.create_queries()

In [16]:
baseline.train_data[:10]

[[275492, 218812],
 [288296, 264093],
 [273286, 293887],
 [57162, 62059],
 [82146, 67997],
 [56777, 61857],
 [169445, 165179],
 [250521, 273893],
 [247266, 241461],
 [36781, 38338]]

In [17]:
bug_train_ids = experiment.get_train_ids(baseline.train_data)

In [18]:
idx = np.random.choice(baseline.bug_ids, 1)[0]
baseline.bug_set[idx]

{'bug_severity': '4\n',
 'bug_status': '1\n',
 'component': '864\n',
 'creation_ts': '2010-10-08 14:07:00 -0400',
 'delta_ts': '2011-05-13 10:58:50 -0400',
 'description': '[CLS] created attachment 1805 ##0 ##9 fix . when using this preference , if i press and hold the step - over key , i see that in my variables views the view updates after every few steps , instead of after every step . the faulty part is the logic that calculate ##s how many updates are supposed to be completed after every step . instead , i tried a different method , where a view listener waits for the " view updates complete " event from the viewer . using this listener in the past was not possible because it would issue multiple such events before the view was actually finished up ##dating . [SEP]',
 'description_segment': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,

### Generating the batch test

In [19]:
"Train ", len(baseline.dup_sets_train)

('Train ', 39339)

In [20]:
import random
random.choice(list(issues_by_buckets))

10640

In [21]:
110647 in experiment.baseline.bug_set

True

In [22]:
%%time

batch_size = 64
batch_size_test = 128

# we want a constant validation group to have a frame of reference for model performance
batch_triplets_valid, valid_input_sample, valid_input_pos, valid_input_neg, valid_sim = experiment.batch_iterator(None, 
                                                                                          baseline.train_data, 
                                                                                          baseline.dup_sets_train,
                                                                                          bug_train_ids,
                                                                                          batch_size_test, 1,
                                                                                          issues_by_buckets)

pos = np.full((1, batch_size_test), 1)
neg = np.full((1, batch_size_test), 0)
valid_sim = np.concatenate([pos, neg], -1)[0]

valid_title_sample_a = np.concatenate([valid_input_sample['title'], valid_input_sample['title']], 0)
valid_title_sample_b = np.concatenate([valid_input_pos['title'], valid_input_neg['title']], 0)
valid_desc_sample_a = np.concatenate([valid_input_sample['description'], valid_input_sample['description']], 0)
valid_desc_sample_b = np.concatenate([valid_input_pos['description'], valid_input_neg['description']], 0)

validation_sample = [valid_title_sample_a, valid_title_sample_b, valid_desc_sample_a, valid_desc_sample_b]

# Max sequence title
MAX_SEQUENCE_LENGTH_T = valid_input_sample['title'].shape[1]
MAX_SEQUENCE_LENGTH_D = valid_input_sample['description'].shape[1]

CPU times: user 471 ms, sys: 0 ns, total: 471 ms
Wall time: 471 ms


In [23]:
len(valid_title_sample_a), len(valid_sim)

(256, 256)

In [24]:
valid_input_sample['title'].shape, valid_input_sample['description'].shape, valid_sim.shape

((128, 20), (128, 20), (256,))

### Validar entrada

In [25]:
# %%time 

#baseline.display_batch(baseline.train_data, baseline.dup_sets_train, bug_train_ids, 5)

## Pre-trained embeddings

Loading pretrained word vectors

### Fasttext

In [26]:
vocab = baseline.load_vocabulary(os.path.join(DIR, 'vocab_embed.pkl'))
#print(np.random.choice(vocab, 10))
# for token in vocab:
#     print(token)

vocabulary loaded


In [27]:
"Total vocabulary: {}".format(len(vocab))

'Total vocabulary: 21175'

In [28]:
def generating_embed(baseline, GLOVE_DIR, EMBEDDING_DIM):
    embeddings_index = {}
    embed_path = os.path.join(GLOVE_DIR, 'glove.42B.300d.txt')
    
    f2 = open(embed_path, 'rb')
    num_lines = sum(1 for line in f2)
    f2.close()
    
    f = open(embed_path, 'rb')
    vocab = baseline.load_vocabulary(os.path.join(baseline.DIR, 'vocab_embed.pkl'))
    vocab_size = len(vocab) 

    # Initialize uniform the vector considering the Tanh activation
    embedding_matrix = np.random.uniform(-1.0, 1.0, (num_lines + vocab_size, EMBEDDING_DIM))
    embedding_matrix[0, :] = np.zeros(EMBEDDING_DIM)

    loop = tqdm(f)
    loop.set_description("Loading Glove")
    i = 0
    for line in loop:
        tokens = line.split()
        word = tokens[0]
        embed = np.asarray(tokens[1:], dtype='float32')
        embeddings_index[word] = embed
        embedding_matrix[i] = embed
        loop.update(1)
    f.close()
    loop.close()

    print('Total %s word vectors in Glove 42B 300d.' % len(embeddings_index))

    loop = tqdm(total=vocab_size)
    loop.set_description('Loading embedding from dataset pretrained')
    
    for word, embed in vocab.items():
        if word in embeddings_index:
            embedding_matrix[i] = embeddings_index[word]
        else:
            embedding_matrix[i] = np.asarray(embed, dtype='float32')
        loop.update(1)
        i+=1
    loop.close()
    baseline.embedding_matrix = embedding_matrix

In [29]:
%%time

generating_embed(baseline, GLOVE_DIR=GLOVE_DIR, EMBEDDING_DIM=EMBEDDING_DIM) # MAX_NB_WORDS=MAX_NB_WORDS

vocabulary loaded



Total 1917494 word vectors in Glove 42B 300d.



CPU times: user 1min 37s, sys: 3.33 s, total: 1min 40s
Wall time: 1min 40s


## Experiment

## Propose

https://github.com/tqtg/DuplicateBugFinder

In [30]:
from keras.initializers import RandomUniform, RandomNormal, Ones

### Embedding layer

In [31]:
from keras.constraints import MaxNorm
from keras.initializers import TruncatedNormal, RandomUniform

# Is missing the padding_idx used in pytorch
# https://pytorch.org/docs/stable/_modules/torch/nn/modules/sparse.html
# https://stackoverflow.com/questions/54824768/rnn-model-gru-of-word2vec-to-regression-not-learning
def embedding_layer(embeddings, num_words, embedding_dim, max_sequence_length, trainable, name):
    embedding_layer = Embedding(num_words,
                                  embedding_dim,
                                  name='embedding_layer_{}'.format(name),
                                  weights=[embeddings],
                                  #input_length=max_sequence_length,
                                  input_length=None,
                                  trainable=trainable)
    return embedding_layer

### DWEN model

In [32]:
from keras.layers import concatenate, Add, Lambda, merge, Average, Maximum, Subtract, \
    Average, GlobalAveragePooling1D, BatchNormalization, Activation
from keras.optimizers import Adam, Nadam
import keras.backend as K

def dwen_feature(title_feature_model, desc_feature_model, \
                  sequence_length_t, sequence_length_d, name):
    bug_t = Input(shape = (sequence_length_t, ), name = 'title_{}'.format(name))
    bug_d = Input(shape = (sequence_length_d, ), name = 'desc_{}'.format(name))
    
    # Embedding feature
    bug_t_feat = title_feature_model(bug_t)
    bug_d_feat = desc_feature_model(bug_d)
    
    bug_t_feat = GlobalAveragePooling1D()(bug_t_feat)
    bug_d_feat = GlobalAveragePooling1D()(bug_d_feat)
    
    #bug_feature_output = Add(name = 'merge_features_{}'.format(name))([bug_i_feat, bug_t_feat, bug_d_feat])
    bug_feature_output = Average(name = 'merge_features_{}'.format(name))([bug_t_feat, bug_d_feat])
    
    bug_feature_model = Model(inputs=[bug_t, bug_d], outputs=[bug_feature_output], name = 'merge_features_{}'.format(name))
    
    return bug_feature_model

def dwen_model(bug_feature_output_a, bug_feature_output_b, name):
    
    inputs = np.concatenate([bug_feature_output_a.input, bug_feature_output_b.input], -1).tolist()
    
    bug_feature_output_a = bug_feature_output_a.output
    bug_feature_output_b = bug_feature_output_b.output
    
    # 2D concatenate feature
    bug_feature_output = concatenate([bug_feature_output_a, bug_feature_output_b])
    
    hidden_layers = 2
    
    # Deep Hidden MLPs
    for _ in range(hidden_layers):
        number_of_units = K.int_shape(bug_feature_output)[1]
        bug_feature_output = Dense(number_of_units // 2)(bug_feature_output)
#         bug_feature_output = BatchNormalization()(bug_feature_output)
        bug_feature_output = Activation('relu')(bug_feature_output)
        #bug_feature_output = Dropout(.5)(bug_feature_output)
    
     # Sigmoid
    output = Dense(1, activation='sigmoid')(bug_feature_output)

    similarity_model = Model(inputs=inputs, outputs=[output], name = 'dwen_output')

    #optimizer = Nadam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=K.epsilon(), schedule_decay=0.01)
    optimizer = Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)

    # setup the optimization process 
    similarity_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
    
    return similarity_model

In [33]:
import _pickle as pickle
def save_loss(result):
    with open(os.path.join(DIR,'{}_log.pkl'.format(METHOD)), 'wb') as f:
        pickle.dump(result, f)
    print("=> result saved!")

In [34]:
# Domain to use
limit_train = int(epochs * freeze_train) # 10% de 1000 , 100 epocas
METHOD = 'baseline_dwen_{}'.format(limit_train)
SAVE_PATH = '{}_preprocessing_{}_feature@number_of_epochs@epochs_64batch({})'.format(PREPROCESSING, METHOD, DOMAIN)
SAVE_PATH_FEATURE = '{}_preprocessing_{}_feature_@number_of_epochs@epochs_64batch({})'.format(PREPROCESSING, METHOD, DOMAIN)

In [35]:
%%time

# Inspired on https://'pastebin.com/TaGFdcBA
# TODO: https://stackoverflow.com/questions/49941903/keras-compute-cosine-distance-between-two-flattened-outputs
keras.backend.clear_session()

# Clear GPU memory
# from numba import cuda
# cuda.select_device(0)
# cuda.close()

# Embeddings
desc_embedding_layer = embedding_layer(embeddings=baseline.embedding_matrix, 
                              num_words=len(baseline.embedding_matrix), 
                              embedding_dim=EMBEDDING_DIM, 
                              max_sequence_length=MAX_SEQUENCE_LENGTH_D, 
                              trainable=False, name='desc')
title_embedding_layer = embedding_layer(embeddings=baseline.embedding_matrix, 
                              num_words=len(baseline.embedding_matrix), 
                              embedding_dim=EMBEDDING_DIM, 
                              max_sequence_length=MAX_SEQUENCE_LENGTH_T, 
                              trainable=False, name='title')

# Similarity model
bug_feature_output_a = dwen_feature(title_embedding_layer, desc_embedding_layer, 
                                    MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'dwen_a')
bug_feature_output_b = dwen_feature(title_embedding_layer, desc_embedding_layer, 
                                    MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'dwen_b')
similarity_model = dwen_model(bug_feature_output_a, bug_feature_output_b, 'dwen')

# cnn_feature_model.summary()
# lstm_feature_model.summary()
similarity_model.summary()


'''
    Experiment
'''
result = { 'train' : [], 'test' : [] }
print("Total of ", limit_train)
for epoch in range(limit_train):
    batch_triplet_train, \
        train_input_sample, train_input_pos, train_input_neg, \
            train_sim = experiment.batch_iterator(None, baseline.train_data, baseline.dup_sets_train, 
                                                  bug_train_ids, batch_size, 1, issues_by_buckets, TRIPLET_HARD=False)
    
    num_batch = train_input_sample['title'].shape[0]
    pos = np.full((1, num_batch), 1)
    neg = np.full((1, num_batch), 0)
    train_sim = np.concatenate([pos, neg], -1)[0]
    
    title_sample_a = np.concatenate([train_input_sample['title'], train_input_sample['title']], 0)
    title_sample_b = np.concatenate([train_input_pos['title'], train_input_neg['title']], 0)
    desc_sample_a = np.concatenate([train_input_sample['description'], train_input_sample['description']], 0)
    desc_sample_b = np.concatenate([train_input_pos['description'], train_input_neg['description']], 0)
    train_batch = [title_sample_a, desc_sample_a, title_sample_b, desc_sample_b]
    
    
    h = similarity_model.train_on_batch(x=train_batch, y=train_sim)
    h_validation = similarity_model.test_on_batch(x=validation_sample, y=valid_sim)
    
    # save results
    result['train'].append(h)
    result['test'].append(h_validation)
    
    if( (epoch+1) % 10 == 0 or (epoch+1 == limit_train) ):
        save_loss(result)
    
    if (epoch+1 == limit_train): #(epoch > 1 and epoch % 10 == 0) or (epoch+1 == epochs):
        recall, exported_rank, debug = experiment.evaluate_validation_test(retrieval, verbose, bug_feature_output_a, issues_by_buckets, 
                                                        bug_train_ids, 'dwen')
        print("Epoch: {} Loss: {:.2f}, Loss_test: {:.2f}, acc: {:.2f}, acc_tets: {:.2f}, recall@25: {:.2f}".format(epoch+1, h[0], h_validation[0],  h[1], h_validation[1], recall))
    else:
        print("Epoch: {} Loss: {:.2f}, Loss_test: {:.2f}, acc: {:.2f}, acc_test: {:.2f}".format(epoch+1, h[0], h_validation[0], h[1], h_validation[1]))
    
    loss = h[0]
    
    if loss < best_loss:
        best_loss = loss
        best_epoch = epoch+1

#experiment.save_model(similarity_model, SAVE_PATH.replace('@number_of_epochs@', str(epochs)))
#experiment.save_model(bug_feature_output_a, SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs)), verbose=1)
print('Best_epoch={}, Best_loss={:.2f}, Recall@25={:.2f}'.format(best_epoch, best_loss, recall))














Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
title_dwen_a (InputLayer)       (None, 20)           0                                            
__________________________________________________________________________________________________
desc_dwen_a (InputLayer)        (None, 20)           0                                            
__________________________________________________________________________________________________
title_dwen_b (InputLayer)       (None, 20)           0                                            
__________________________________________________________________________________________________
desc_dwen_b (InputLayer)        (None, 20)           0                                            
__

Epoch: 8 Loss: 0.68, Loss_test: 0.69, acc: 0.66, acc_test: 0.53
Epoch: 9 Loss: 0.68, Loss_test: 0.69, acc: 0.57, acc_test: 0.54
=> result saved!
Epoch: 10 Loss: 0.69, Loss_test: 0.69, acc: 0.54, acc_test: 0.54
Epoch: 11 Loss: 0.67, Loss_test: 0.69, acc: 0.64, acc_test: 0.55
Epoch: 12 Loss: 0.67, Loss_test: 0.69, acc: 0.59, acc_test: 0.55
Epoch: 13 Loss: 0.67, Loss_test: 0.69, acc: 0.62, acc_test: 0.55
Epoch: 14 Loss: 0.68, Loss_test: 0.69, acc: 0.61, acc_test: 0.53
Epoch: 15 Loss: 0.67, Loss_test: 0.69, acc: 0.55, acc_test: 0.55
Epoch: 16 Loss: 0.68, Loss_test: 0.69, acc: 0.55, acc_test: 0.53
Epoch: 17 Loss: 0.68, Loss_test: 0.69, acc: 0.55, acc_test: 0.53
Epoch: 18 Loss: 0.67, Loss_test: 0.69, acc: 0.63, acc_test: 0.53
Epoch: 19 Loss: 0.67, Loss_test: 0.69, acc: 0.59, acc_test: 0.54
=> result saved!
Epoch: 20 Loss: 0.67, Loss_test: 0.69, acc: 0.59, acc_test: 0.54
Epoch: 21 Loss: 0.67, Loss_test: 0.69, acc: 0.55, acc_test: 0.55
Epoch: 22 Loss: 0.66, Loss_test: 0.69, acc: 0.59, acc_test

In [36]:
EXPORT_RANK_PATH = os.path.join(DIR, 'exported_rank_{}.txt'.format(METHOD))
EXPORT_RANK_PATH

'data/processed/eclipse/bert/exported_rank_baseline_dwen_100.txt'

In [37]:
with open(EXPORT_RANK_PATH, 'w') as file_out:
    for row in exported_rank:
        file_out.write(row + "\n")

In [38]:
experiment.save_model(similarity_model, SAVE_PATH.replace('@number_of_epochs@', str(limit_train)))
experiment.save_model(bug_feature_output_a, SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(limit_train)), verbose=1)

Saved model 'modelos/model_bert_preprocessing_baseline_dwen_100_feature_100epochs_64batch(eclipse).h5' to disk


In [39]:
len(result['train']), len(result['test'])

(100, 100)

In [40]:
model = similarity_model.get_layer('dense_3')
output = model.output
inputs = similarity_model.inputs
model = Model(inputs = inputs, outputs = output, name = 'Similarity_Model')

# setup the optimization process 
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
title_dwen_a (InputLayer)       (None, 20)           0                                            
__________________________________________________________________________________________________
desc_dwen_a (InputLayer)        (None, 20)           0                                            
__________________________________________________________________________________________________
title_dwen_b (InputLayer)       (None, 20)           0                                            
__________________________________________________________________________________________________
desc_dwen_b (InputLayer)        (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_

In [41]:
# Domain to use
METHOD = 'baseline_dwen_{}'.format(epochs)
SAVE_PATH = '{}_preprocessing_{}_feature@number_of_epochs@epochs_64batch({})'.format(PREPROCESSING, METHOD, DOMAIN)
SAVE_PATH_FEATURE = '{}_preprocessing_{}_feature_@number_of_epochs@epochs_64batch({})'.format(PREPROCESSING, METHOD, DOMAIN)

In [42]:
end_train = epochs - limit_train
for epoch in range(limit_train, end_train):
    batch_triplet_train, \
        train_input_sample, train_input_pos, train_input_neg, \
            train_sim = experiment.batch_iterator(None, baseline.train_data, baseline.dup_sets_train, 
                                                  bug_train_ids, batch_size, 1, issues_by_buckets, TRIPLET_HARD=False)
    
    num_batch = train_input_sample['title'].shape[0]
    pos = np.full((1, num_batch), 1)
    neg = np.full((1, num_batch), 0)
    train_sim = np.concatenate([pos, neg], -1)[0]
    
    title_sample_a = np.concatenate([train_input_sample['title'], train_input_sample['title']], 0)
    title_sample_b = np.concatenate([train_input_pos['title'], train_input_neg['title']], 0)
    desc_sample_a = np.concatenate([train_input_sample['description'], train_input_sample['description']], 0)
    desc_sample_b = np.concatenate([train_input_pos['description'], train_input_neg['description']], 0)
    train_batch = [title_sample_a, desc_sample_a, title_sample_b, desc_sample_b]
    
    
    h = similarity_model.train_on_batch(x=train_batch, y=train_sim)
    h_validation = similarity_model.test_on_batch(x=validation_sample, y=valid_sim)
    
    # save results
    result['train'].append(h)
    result['test'].append(h_validation)
    
    if( (epoch+1) % 10 == 0 or (epoch+1 == limit_train) ):
        save_loss(result)
    
    if (epoch+1 == epochs): #(epoch > 1 and epoch % 10 == 0) or (epoch+1 == epochs):
        recall, exported_rank, debug = experiment.evaluate_validation_test(retrieval, verbose, bug_feature_output_a, issues_by_buckets, bug_train_ids, 'dwen')
        print("Epoch: {} Loss: {:.2f}, Loss_test: {:.2f}, acc: {:.2f}, acc_tets: {:.2f}, recall@25: {:.2f}".format(epoch+1, h[0], h_validation[0],  h[1], h_validation[1], recall))
    else:
        print("Epoch: {} Loss: {:.2f}, Loss_test: {:.2f}, acc: {:.2f}, acc_test: {:.2f}".format(epoch+1, h[0], h_validation[0], h[1], h_validation[1]))

Epoch: 101 Loss: 0.59, Loss_test: 0.72, acc: 0.64, acc_test: 0.54
Epoch: 102 Loss: 0.56, Loss_test: 0.73, acc: 0.71, acc_test: 0.54
Epoch: 103 Loss: 0.58, Loss_test: 0.72, acc: 0.63, acc_test: 0.54
Epoch: 104 Loss: 0.62, Loss_test: 0.71, acc: 0.61, acc_test: 0.57
Epoch: 105 Loss: 0.60, Loss_test: 0.72, acc: 0.65, acc_test: 0.56
Epoch: 106 Loss: 0.58, Loss_test: 0.71, acc: 0.64, acc_test: 0.55
Epoch: 107 Loss: 0.58, Loss_test: 0.72, acc: 0.70, acc_test: 0.54
Epoch: 108 Loss: 0.57, Loss_test: 0.73, acc: 0.72, acc_test: 0.55
Epoch: 109 Loss: 0.65, Loss_test: 0.72, acc: 0.60, acc_test: 0.54
=> result saved!
Epoch: 110 Loss: 0.62, Loss_test: 0.71, acc: 0.61, acc_test: 0.57
Epoch: 111 Loss: 0.53, Loss_test: 0.71, acc: 0.72, acc_test: 0.54
Epoch: 112 Loss: 0.58, Loss_test: 0.71, acc: 0.66, acc_test: 0.54
Epoch: 113 Loss: 0.58, Loss_test: 0.71, acc: 0.68, acc_test: 0.58
Epoch: 114 Loss: 0.54, Loss_test: 0.72, acc: 0.72, acc_test: 0.56
Epoch: 115 Loss: 0.61, Loss_test: 0.72, acc: 0.66, acc_test

Epoch: 223 Loss: 0.53, Loss_test: 0.72, acc: 0.74, acc_test: 0.55
Epoch: 224 Loss: 0.59, Loss_test: 0.72, acc: 0.69, acc_test: 0.54
Epoch: 225 Loss: 0.54, Loss_test: 0.72, acc: 0.70, acc_test: 0.55
Epoch: 226 Loss: 0.56, Loss_test: 0.72, acc: 0.67, acc_test: 0.55
Epoch: 227 Loss: 0.49, Loss_test: 0.73, acc: 0.78, acc_test: 0.54
Epoch: 228 Loss: 0.53, Loss_test: 0.73, acc: 0.70, acc_test: 0.55
Epoch: 229 Loss: 0.57, Loss_test: 0.72, acc: 0.71, acc_test: 0.54
=> result saved!
Epoch: 230 Loss: 0.51, Loss_test: 0.72, acc: 0.75, acc_test: 0.55
Epoch: 231 Loss: 0.55, Loss_test: 0.72, acc: 0.70, acc_test: 0.55
Epoch: 232 Loss: 0.50, Loss_test: 0.72, acc: 0.76, acc_test: 0.54
Epoch: 233 Loss: 0.50, Loss_test: 0.72, acc: 0.76, acc_test: 0.54
Epoch: 234 Loss: 0.53, Loss_test: 0.72, acc: 0.74, acc_test: 0.54
Epoch: 235 Loss: 0.47, Loss_test: 0.72, acc: 0.73, acc_test: 0.54
Epoch: 236 Loss: 0.48, Loss_test: 0.73, acc: 0.77, acc_test: 0.55
Epoch: 237 Loss: 0.52, Loss_test: 0.73, acc: 0.76, acc_test

Epoch: 345 Loss: 0.41, Loss_test: 0.71, acc: 0.84, acc_test: 0.58
Epoch: 346 Loss: 0.50, Loss_test: 0.71, acc: 0.76, acc_test: 0.57
Epoch: 347 Loss: 0.50, Loss_test: 0.70, acc: 0.75, acc_test: 0.58
Epoch: 348 Loss: 0.46, Loss_test: 0.70, acc: 0.75, acc_test: 0.59
Epoch: 349 Loss: 0.48, Loss_test: 0.71, acc: 0.75, acc_test: 0.58
=> result saved!
Epoch: 350 Loss: 0.56, Loss_test: 0.71, acc: 0.68, acc_test: 0.58
Epoch: 351 Loss: 0.45, Loss_test: 0.70, acc: 0.79, acc_test: 0.56
Epoch: 352 Loss: 0.52, Loss_test: 0.70, acc: 0.75, acc_test: 0.55
Epoch: 353 Loss: 0.51, Loss_test: 0.70, acc: 0.75, acc_test: 0.56
Epoch: 354 Loss: 0.51, Loss_test: 0.71, acc: 0.79, acc_test: 0.55
Epoch: 355 Loss: 0.44, Loss_test: 0.72, acc: 0.80, acc_test: 0.56
Epoch: 356 Loss: 0.48, Loss_test: 0.73, acc: 0.77, acc_test: 0.55
Epoch: 357 Loss: 0.49, Loss_test: 0.74, acc: 0.78, acc_test: 0.56
Epoch: 358 Loss: 0.42, Loss_test: 0.74, acc: 0.80, acc_test: 0.55
Epoch: 359 Loss: 0.51, Loss_test: 0.73, acc: 0.77, acc_test

Epoch: 467 Loss: 0.45, Loss_test: 0.73, acc: 0.77, acc_test: 0.56
Epoch: 468 Loss: 0.41, Loss_test: 0.72, acc: 0.76, acc_test: 0.57
Epoch: 469 Loss: 0.36, Loss_test: 0.72, acc: 0.84, acc_test: 0.58
=> result saved!
Epoch: 470 Loss: 0.62, Loss_test: 0.72, acc: 0.66, acc_test: 0.56
Epoch: 471 Loss: 0.57, Loss_test: 0.72, acc: 0.71, acc_test: 0.57
Epoch: 472 Loss: 0.51, Loss_test: 0.72, acc: 0.74, acc_test: 0.57
Epoch: 473 Loss: 0.54, Loss_test: 0.73, acc: 0.77, acc_test: 0.57
Epoch: 474 Loss: 0.46, Loss_test: 0.72, acc: 0.77, acc_test: 0.55
Epoch: 475 Loss: 0.39, Loss_test: 0.72, acc: 0.84, acc_test: 0.56
Epoch: 476 Loss: 0.46, Loss_test: 0.72, acc: 0.78, acc_test: 0.53
Epoch: 477 Loss: 0.51, Loss_test: 0.72, acc: 0.74, acc_test: 0.55
Epoch: 478 Loss: 0.47, Loss_test: 0.72, acc: 0.77, acc_test: 0.57
Epoch: 479 Loss: 0.39, Loss_test: 0.73, acc: 0.80, acc_test: 0.58
=> result saved!
Epoch: 480 Loss: 0.47, Loss_test: 0.72, acc: 0.74, acc_test: 0.57
Epoch: 481 Loss: 0.41, Loss_test: 0.72, ac

Epoch: 589 Loss: 0.44, Loss_test: 0.71, acc: 0.77, acc_test: 0.57
=> result saved!
Epoch: 590 Loss: 0.50, Loss_test: 0.72, acc: 0.72, acc_test: 0.58
Epoch: 591 Loss: 0.46, Loss_test: 0.72, acc: 0.80, acc_test: 0.58
Epoch: 592 Loss: 0.38, Loss_test: 0.72, acc: 0.88, acc_test: 0.59
Epoch: 593 Loss: 0.49, Loss_test: 0.72, acc: 0.78, acc_test: 0.60
Epoch: 594 Loss: 0.54, Loss_test: 0.73, acc: 0.67, acc_test: 0.57
Epoch: 595 Loss: 0.40, Loss_test: 0.73, acc: 0.83, acc_test: 0.57
Epoch: 596 Loss: 0.54, Loss_test: 0.72, acc: 0.74, acc_test: 0.60
Epoch: 597 Loss: 0.51, Loss_test: 0.72, acc: 0.72, acc_test: 0.62
Epoch: 598 Loss: 0.46, Loss_test: 0.71, acc: 0.78, acc_test: 0.62
Epoch: 599 Loss: 0.39, Loss_test: 0.71, acc: 0.83, acc_test: 0.60
=> result saved!
Epoch: 600 Loss: 0.37, Loss_test: 0.71, acc: 0.84, acc_test: 0.59
Epoch: 601 Loss: 0.47, Loss_test: 0.71, acc: 0.78, acc_test: 0.58
Epoch: 602 Loss: 0.45, Loss_test: 0.71, acc: 0.80, acc_test: 0.56
Epoch: 603 Loss: 0.41, Loss_test: 0.71, ac

Epoch: 711 Loss: 0.49, Loss_test: 0.71, acc: 0.76, acc_test: 0.61
Epoch: 712 Loss: 0.39, Loss_test: 0.71, acc: 0.81, acc_test: 0.62
Epoch: 713 Loss: 0.39, Loss_test: 0.73, acc: 0.85, acc_test: 0.56
Epoch: 714 Loss: 0.38, Loss_test: 0.78, acc: 0.79, acc_test: 0.57
Epoch: 715 Loss: 0.42, Loss_test: 0.76, acc: 0.78, acc_test: 0.57
Epoch: 716 Loss: 0.48, Loss_test: 0.73, acc: 0.78, acc_test: 0.61
Epoch: 717 Loss: 0.38, Loss_test: 0.74, acc: 0.80, acc_test: 0.61
Epoch: 718 Loss: 0.42, Loss_test: 0.76, acc: 0.81, acc_test: 0.60
Epoch: 719 Loss: 0.37, Loss_test: 0.75, acc: 0.82, acc_test: 0.62
=> result saved!
Epoch: 720 Loss: 0.53, Loss_test: 0.73, acc: 0.70, acc_test: 0.57
Epoch: 721 Loss: 0.49, Loss_test: 0.79, acc: 0.75, acc_test: 0.57
Epoch: 722 Loss: 0.47, Loss_test: 0.82, acc: 0.74, acc_test: 0.54
Epoch: 723 Loss: 0.46, Loss_test: 0.78, acc: 0.78, acc_test: 0.57
Epoch: 724 Loss: 0.41, Loss_test: 0.72, acc: 0.79, acc_test: 0.56
Epoch: 725 Loss: 0.44, Loss_test: 0.71, acc: 0.81, acc_test

Epoch: 833 Loss: 0.42, Loss_test: 0.70, acc: 0.78, acc_test: 0.61
Epoch: 834 Loss: 0.37, Loss_test: 0.71, acc: 0.80, acc_test: 0.59
Epoch: 835 Loss: 0.41, Loss_test: 0.73, acc: 0.81, acc_test: 0.57
Epoch: 836 Loss: 0.40, Loss_test: 0.72, acc: 0.78, acc_test: 0.59
Epoch: 837 Loss: 0.39, Loss_test: 0.70, acc: 0.84, acc_test: 0.62
Epoch: 838 Loss: 0.40, Loss_test: 0.72, acc: 0.84, acc_test: 0.62
Epoch: 839 Loss: 0.41, Loss_test: 0.72, acc: 0.78, acc_test: 0.60
=> result saved!
Epoch: 840 Loss: 0.40, Loss_test: 0.71, acc: 0.83, acc_test: 0.63
Epoch: 841 Loss: 0.46, Loss_test: 0.70, acc: 0.82, acc_test: 0.64
Epoch: 842 Loss: 0.43, Loss_test: 0.72, acc: 0.81, acc_test: 0.59
Epoch: 843 Loss: 0.39, Loss_test: 0.72, acc: 0.81, acc_test: 0.61
Epoch: 844 Loss: 0.43, Loss_test: 0.70, acc: 0.77, acc_test: 0.62
Epoch: 845 Loss: 0.45, Loss_test: 0.71, acc: 0.76, acc_test: 0.60
Epoch: 846 Loss: 0.36, Loss_test: 0.71, acc: 0.84, acc_test: 0.60
Epoch: 847 Loss: 0.47, Loss_test: 0.71, acc: 0.75, acc_test

In [43]:
len(result['train']), len(result['test'])

(900, 900)

In [44]:
encoded = model.get_layer('merge_features_dwen_a')
output = encoded.output
inputs = similarity_model.inputs[:-2]
bug_feature_output_a = Model(inputs = inputs, outputs = output, name = 'Similarity_Model')

In [45]:
inputs

[<tf.Tensor 'title_dwen_a:0' shape=(?, 20) dtype=float32>,
 <tf.Tensor 'desc_dwen_a:0' shape=(?, 20) dtype=float32>]

In [46]:
SAVE_PATH.replace('@number_of_epochs@', str(epochs))

'bert_preprocessing_baseline_dwen_1000_feature1000epochs_64batch(eclipse)'

In [47]:
experiment.save_model(model, SAVE_PATH.replace('@number_of_epochs@', str(epochs)))
experiment.save_model(bug_feature_output_a, SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs)), verbose=1)
"Model saved"

Saved model 'modelos/model_bert_preprocessing_baseline_dwen_1000_feature_1000epochs_64batch(eclipse).h5' to disk


'Model saved'

In [48]:
recall, exported_rank, debug = experiment.evaluate_validation_test(retrieval, verbose, bug_feature_output_a, issues_by_buckets, bug_train_ids, 'dwen')
print("Epoch: {} Loss: {:.2f}, Loss_test: {:.2f}, acc: {:.2f}, acc_tets: {:.2f}, recall@25: {:.2f}".format(epoch+1, h[0], h_validation[0],  h[1], h_validation[1], recall))

Epoch: 900 Loss: 0.34, Loss_test: 0.74, acc: 0.84, acc_tets: 0.61, recall@25: 0.16


In [49]:
exported_rank[:20]

['2:15392,9779,94|387108:0.47366011142730713,396638:0.42618709802627563,271242:0.42618048191070557,307723:0.39547091722488403,167697:0.39304739236831665,45583:0.3848111629486084,208090:0.3832539916038513,171:0.35479652881622314,153780:0.3529524803161621,14751:0.3467496633529663,274173:0.32306307554244995,317885:0.31425559520721436,2939:0.30586034059524536,150663:0.2966539263725281,359474:0.29371416568756104,251907:0.2873278260231018,227324:0.27639782428741455,27244:0.27262425422668457,42884:0.26375865936279297,23728:0.260738730430603,347699:0.2584395408630371,202311:0.2565629482269287,394703:0.24644291400909424,47020:0.243586003780365,386952:0.24335312843322754,67074:0.2412591576576233,64081:0.24061650037765503,32202:0.23618894815444946,23264:0.23561573028564453',
 '393232:393282,390667,383388|393282:0.5613614618778229,146301:0.2514530420303345,196411:0.24373668432235718,102885:0.2353416085243225,64146:0.23315370082855225,82121:0.23052126169204712,164395:0.22540080547332764,148834:0.22

### Retrieval evaluation

In [50]:
print("Total of queries:", len(retrieval.test))

Total of queries: 16995


#### Getting the model trained

In [51]:
SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs))

'bert_preprocessing_baseline_dwen_1000_feature_1000epochs_64batch(eclipse)'

In [52]:
bug_feature_output_a.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
title_dwen_a (InputLayer)       (None, 20)           0                                            
__________________________________________________________________________________________________
desc_dwen_a (InputLayer)        (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_layer_title (Embeddin (None, 20, 300)      581600700   title_dwen_a[0][0]               
__________________________________________________________________________________________________
embedding_layer_desc (Embedding (None, 20, 300)      581600700   desc_dwen_a[0][0]                
__________________________________________________________________________________________________
global_ave

In [53]:
EXPORT_RANK_PATH = os.path.join(DIR, 'exported_rank_{}.txt'.format(METHOD))
EXPORT_RANK_PATH

'data/processed/eclipse/bert/exported_rank_baseline_dwen_1000.txt'

In [54]:
with open(EXPORT_RANK_PATH, 'w') as file_out:
    for row in exported_rank:
        file_out.write(row + "\n")

In [55]:
report = experiment.evaluation.evaluate(EXPORT_RANK_PATH)
report

{'1 - recall_at_5': 0.13,
 '2 - recall_at_10': 0.14,
 '3 - recall_at_15': 0.15,
 '4 - recall_at_20': 0.16,
 '5 - recall_at_25': 0.16}